In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, Activation
# from tensorflow.keras.layers import LeakyReLU, ELU , ReLU
# from tensorflow.keras.activations import relu
import sys
import numpy as np
import os
import matplotlib.pyplot as plt
import skimage
import skimage.measure as skim
import copy
import nibabel as nib
# plt.style.available

In [ ]:
organ = 'PAROTID_LT'; train_test = 'train_odd'
folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/'
# ctour_folder = folder + '{}/{}/'.format(organ, train_test)
ctour_folder = folder + '{}/complete_arrays/'.format(organ)
# regt = np.load(folder+'PAROTID_LT/complete_arrays/test_even_PAROTID_LT_registration_track.npy', mmap_mode = 'c')
ref_slices = np.load(ctour_folder+'{}_{}_multiple_ref_contour_image_array.npy'.format(train_test, organ), mmap_mode = 'r').astype(np.float32)
reg_slices = np.load(ctour_folder+'{}_{}_multiple_def_contour_image_array.npy'.format(train_test, organ), mmap_mode = 'r').astype(np.float32)

# ref_slices = np.load(ctour_folder+'ref_contour_image_array_{}_{}_0-544.npy'.format(train_test, organ), mmap_mode = 'r').astype(np.float32)
# reg_slices = np.load(ctour_folder+'def_contour_image_array_{}_{}_0-544.npy'.format(train_test, organ), mmap_mode = 'r').astype(np.float32)

# print(sum(regt[:11, -1]))
# print(len(slices))
# print(slices.shape)
# print(type(slices[0,0,0]))
for i in range(len(ref_slices)):
#     print(i.shape)
    print(i)
    plt.imshow(ref_slices[i]) #     plt.imshow(i[128:-128,128:-128])
    plt.show()
    plt.imshow(reg_slices[i]) #     plt.imshow(i[128:-128,128:-128])
    plt.show()

In [ ]:
# view registered images
fr='Flo_251_Ref_576'
organ='PAROTID_LT'
fp='/hepgpu9-data1/sbridger/Auto_Registration/registration_store/new_registration/train_odd/'+\
'{}/Def_{}.nii'.format(fr, fr)
fp_contour='/hepgpu9-data1/sbridger/Auto_Registration/registration_store/new_registration/train_odd/'+\
'{}/Def_{}_{}.nii.gz'.format(fr,fr,organ)

# Def_Flo_661_Ref_479.nii

# fr='Flo_251_Ref_661'
# organ='PAROTID_LT'
# fp='/hepgpu3-data1/denisp/CNN_test/{}/Def_{}.nii.gz'.format(fr, fr)
# fp_contour='/hepgpu3-data1/denisp/CNN_test/{}/Def_{}_{}.nii.gz'.format(fr,fr,organ)

img_data = nib.load(fp)
reg_img = img_data.get_fdata()
contour_data = nib.load(fp_contour)
reg_contour = contour_data.get_fdata()
# print(reg_img.shape)
cut=128
for i in range(reg_img.shape[-1]):
#     print(reg_img[100,100,i])
    if np.any(reg_contour[:,:,i]):
        print(i)
        plt.imshow(reg_img[cut:-cut,cut:-cut,i])
        plt.show()

In [ ]:
organ = 'PAROTID_LT'

invalid_vi = np.load('PAROTID_LT_validate_invalid_DVF_slice_indexes.npy', mmap_mode='r')
invalid_ti = np.load('PAROTID_LT_train_invalid_DVF_slice_indexes.npy', mmap_mode='r')

if organ=='BRAIN_STEM':
    n_train_max = 11273
    n_validate_max = 6154
if organ=='PAROTID_LT':
    n_train_max = 13394
    n_validate_max = 7762
    
# Indexes for new images
entire_invalid_idxs = np.concatenate((invalid_vi,invalid_ti+n_validate_max))

folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/'+organ+'/'
prefix = 'complete_arrays/COMPLETE_' + organ + '_'

reg_track_load = np.load(folder+prefix+'registration_track.npy')
rt_new=np.array_split(reg_track_load, [len(reg_track_load),4])[0]

c=0
for count, i in enumerate(rt_new[:34]):
#     print(i)
    c+=i[-1]
print(c)
print(rt_new[33])
# print(entire_invalid_idxs[:100])

flo_no = '077'; ref_no = '607'
dvf_invalid = nib.load('/hepgpu9-data1/sbridger/Auto_Registration/registration_store/'+\
                       'train_even/Flo_{}_Ref_{}/Disp_Vector_Field.nii'.format(flo_no, ref_no)).get_fdata()
print(dvf_invalid.shape)

In [ ]:
dice_array_name = 'multiple_dice_array_PAROTID_LT10-20_April-02-2019'
patient_array_name = 'multiple_patient_image_array_PAROTID_LT10-20_April-02-2019'

filep='/hepgpu9-data1/callum/DeepRegQA/data/arrays_3D_z_direction_25Mar/arrays_3D_z_direction/PAROTID_LT/'
rt_FP='registration_track_25Mar_PAROTID_LT.npy'
reg_trackL = np.load(filep+rt_FP)

filepathP='/hepgpu9-data1/callum/DeepRegQA/data/processed/PAROTID_LT/registration_track_5April_PAROTID_LT.npy'
# reg_trackL = np.load(filepathP)
n=30
rt = np.array_split(reg_trackL, [len(reg_trackL),4])[0]#[:n]

organ = 'PAROTID_LT'
folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/'+organ+'/'
prefix = 'complete_arrays/COMPLETE_' + organ + '_'

reg_track_load = np.load(folder+prefix+'registration_track.npy')
rt_new=np.array_split(reg_track_load, [len(reg_track_load),4])[0]#[:n]
# print(rt[:,:2])

def transf(rt):
    return int(rt[0])*np.sqrt(int(rt[1])-0.5)

RT=[]
sum_=0
for i in rt:
    AnOp = transf(i)
    RT_ = [AnOp, *i[:2]] 
    sum_+=int(i[2])
#     RT.append(RT_)
    RT.append(AnOp)
    
rt_new_=[]
for i in rt_new:        
    AnOp = transf(i)
    RT_ = [AnOp, *i[:2]]
#     rt_new_.append(RT_)
    rt_new_.append(AnOp)

print(len(RT))
print(len(set(RT)))
print(len(rt_new_))
print(len(set(rt_new_)))
set_intersection = set(RT).intersection(set(rt_new_))
print(len(set_intersection))

# print(rt[0])

cross_matching_pairs, sum_of_img_slices = [], 0
for idx, i in enumerate(rt_new):
    transf_val = transf(i)
    sum_of_img_slices+=int(i[-1])
    if np.isin(transf_val, sorted(set_intersection)):
        n_slice_start = sum_of_img_slices-int(i[-1])
        cross_matching_pairs.append([idx, n_slice_start, *i])
        

print(cross_matching_pairs)

new_metric = np.load(folder+prefix+'multiple_msd_3D_array.npy', mmap_mode='r')
metric = np.load(filep+'msd_array_3D_25Mar_PAROTID_LT.npy', mmap_mode='r')

# print(new_metric[139:139+14])
# print(metric[:28])

new_imgs = np.load(folder+prefix+'multiple_patient_image_array.npy', mmap_mode='r')#.astype(np.float16)
imgs = np.load(filep+'image_array_3D_25Mar_PAROTID_LT.npy', mmap_mode='r')#.astype(np.float16)
# print(type(imgs))


# print(cross_matching_pairs)
matching_new_indexes = []
for i in cross_matching_pairs:
#     print(i)
    indexes = np.arange(int(i[1]), int(i[1])+int(i[-1]))
    matching_new_indexes.append(indexes)
#     print(indexes)

rt_new_matching_indexes = np.concatenate(matching_new_indexes).ravel()

# print(rt_new_matching_indexes)

invalid_vi = np.load('PAROTID_LT_validate_invalid_DVF_slice_indexes.npy', mmap_mode='r')
invalid_ti = np.load('PAROTID_LT_train_invalid_DVF_slice_indexes.npy', mmap_mode='r')

if organ=='BRAIN_STEM':
    n_train_max = 11273
    n_validate_max = 6154
if organ=='PAROTID_LT':
    n_train_max = 13394
    n_validate_max = 7762
    
# Indexes for new images
entire_invalid_idxs = np.concatenate((invalid_vi,invalid_ti+n_validate_max))

check_matching_pairs_invalid_DVF = np.isin(rt_new_matching_indexes, entire_invalid_idxs)

indexes_to_compare_dvfs = []

for count, i in enumerate(check_matching_pairs_invalid_DVF):
    if i==True:
        indexes_to_compare_dvfs.append(rt_new_matching_indexes[count])

const_dvfs=[]
for i in indexes_to_compare_dvfs:
#     print(imgs[i,:,:,1:3])
    dvf_x, dvf_y = new_imgs[i,:,:,1], new_imgs[i,:,:,2]
    if np.min(dvf_x)==np.max(dvf_x) and np.min(dvf_y)==np.max(dvf_y):
#         print(i)
        const_dvfs.append(i)
#         print(dvf_x)
#         print(dvf_y)


# print(len(indexes_to_compare_dvfs),len(const_dvfs))
# print(const_dvfs)
# print(cross_matching_pairs)
# for i in const_dvfs:
print(rt[:10])
print(rt_new[:10])
    
    
def show_xy_slices(data, n_slice_lower, n_slice_upper, steps):
    for i in np.arange(n_slice_lower, n_slice_upper, steps):
        print('Slice number {}'.format(i))
        plt.imshow(data[i,:,:,0], interpolation='none')
        plt.xlabel('x')
        plt.ylabel('y', rotation=0)
        plt.show()


# show_xy_slices(new_imgs,0,14,1)
# show_xy_slices(imgs,0,31,1)

def show_xy_slices_pair(data1, data2, n_slice_lower, n_slice_upper, steps):
    for i in np.arange(n_slice_lower, n_slice_upper, steps):
        print('Slice number {}'.format(i))
        plt.imshow(data1[i,:,:,0], interpolation='none')
        plt.xlabel('x')
        plt.ylabel('y', rotation=0)
        plt.show()
        plt.imshow(data2[i,:,:,0], interpolation='none')
        plt.xlabel('x')
        plt.ylabel('y', rotation=0)
        plt.show()
    
# show_xy_slices_pair(imgs, new_imgs, 0, 14, 1)

rt_indexes_const_dvfs = []
for i in const_dvf_pairs:
    for idx_rt, j in enumerate(rt):
        if i[0]==int(j[0]) and i[1]==int(j[1]):
            rt_indexes_const_dvfs.append(find_slice_idxs(rt, idx_rt))
            
def find_slice_idxs(rt, idx):
    slice_sum=0
    for index_count, i in enumerate(rt):
        if idx>index_count:
            slice_sum+=int(i[-1])
            
    return np.arange(slice_sum, slice_sum+int(rt[idx,-1]))

old_rt_indexes_const_dvfs = sorted(np.concatenate(rt_indexes_const_dvfs).ravel())
# print((old_rt_indexes_const_dvfs))

num=0
for count, i in enumerate(imgs[old_rt_indexes_const_dvfs]):
    dvf_x, dvf_y = i[:,:,1], i[:,:,2]
    if np.min(dvf_x)==np.max(dvf_x) and np.min(dvf_y)==np.max(dvf_y):
        num+=1
#         if np.min(dvf_x)!=0 and np.min(dvf_y)!=0:
#         print(dvf_x, dvf_y)
print(num, len(const_dvf_pairs))

In [ ]:
print(entire_invalid_idxs[:100])
print(reg_track_invalid[:100])

bools = np.isin(np.array(reg_track_invalid)[:,0], entire_invalid_idxs, invert=False)

In [ ]:
organ = 'PAROTID_LT'
folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/'+organ+'/'
prefix = 'complete_arrays/COMPLETE_' + organ + '_'
# reg_track = np.load(folder+'test_odd/registration_track_test_odd_'+organ+'_0-544.npy')
# reg_track = np.load(folder+'test_odd/registration_track_test_odd_'+organ+'_545-1080.npy')
# reg_track = np.load(folder+'train_even/registration_track_train_even_'+organ+'_0-544.npy')
# reg_track = np.load(folder+'train_even/registration_track_train_even_'+organ+'_545-1078.npy')
reg_track_load = np.load(folder+prefix+'registration_track.npy')
# reg_track = np.load(folder+'complete_arrays/train_even_'+organ+'_registration_track.npy')
# reg_track = np.load(folder+'complete_arrays/test_odd_'+organ+'_registration_track.npy')
# odd: 545-1080  even: 545-1078 both: 0-544
# Reg track shaped like [flo, ref, n_metrics, n_slices] (n_metrics=n_slices)
    
rt=np.array_split(reg_track_load, [len(reg_track_load),4])[0]
# print(rt[:50])

In [ ]:
# print(rt)
rtrack_sum=0
rt_indexes=[]
for i in rt:
    rtrack_sum+=i[-1]
    for j in range(int(i[-1])):
        rt_indexes.append([*i[:3]])

# print(len(rt_indexes))
rt_indexes=np.array(rt_indexes)
slice_contour_array = np.empty(shape=(len(rt_indexes),512,512,3), dtype=np.float32)

# for i in rt_indexes:
flos, refs = rt[:,0], rt[:,1]#rt_indexes[:,0], rt_indexes[:,1]

def yield_correct_slice(img_data, index, dtype):
    z_slice = img_data.get_fdata()[:,:,index].astype(dtype)
    return z_slice

n_contour_slices=0
array=[]
data_type=np.float32
it=0
for f,r in zip(flos,refs):
    print('iteration ',it)
    flo_int=int(f)
    ref_int=int(r)

    ref_str = str(ref_int) if ref_int>=100 else '0' + str(ref_int)
    if ref_int<10:
        ref_str = '00' + str(ref_int)
    flo_str = str(flo_int) if flo_int>=100 else '0' + str(flo_int)
    if flo_int<10:
        flo_str = '00' + str(flo_int)

    ref_img_filepath = '/hepgpu9-data1/sbridger/94_patient_data/patient_filepath/{}/NoBackground_0522c0{}.nii.gz'\
    .format(ref_str, ref_str)
    ref_image = nib.load(ref_img_filepath)
#     for i in range(len(ref_image)):
        
#     ref_z_slice = yield_correct_slice(ref_image, 0, data_type)
#         if np.any(ref_z_slice)
    
    number_of_ref_img_slices=ref_image.get_fdata().shape[-1]

    flo_img_filepath = '/hepgpu9-data1/sbridger/94_patient_data/patient_filepath/{}/NoBackground_0522c0{}.nii.gz'\
    .format(flo_str, flo_str)
    flo_image = nib.load(flo_img_filepath)
#     for i in 
#     flo_z_slice = yield_correct_slice(flo_image, 0, data_type)
    
    number_of_flo_img_slices=flo_image.get_fdata().shape[-1]

#     ref_contour_filepath = '/hepgpu9-data1/sbridger/94_patient_data/patient_filepath/{}/{}.nii.gz'\
#     .format(ref_str, organ)
#     ref_contour_image = nib.load(ref_contour_filepath)
    
#     contour_count=0
#     for i in range(ref_contour_image.shape[-1]):
#         ref_z_contour_slice = yield_correct_slice(ref_contour_image, i, data_type)
#         if np.any(ref_z_contour_slice):
#             contour_count+=1
            
    
#     number_of_ref_contour_slices=ref_contour_image.get_fdata().shape[-1]
    
#     flo_contour_filepath = '/hepgpu9-data1/sbridger/94_patient_data/patient_filepath/{}/{}.nii.gz'\
#     .format(flo_str, organ)
#     flo_contour_image = nib.load(flo_contour_filepath)
#     flo_z_contour_slice = yield_correct_slice(flo_contour_image, 0, data_type)
    
#     number_of_flo_contour_slices=flo_contour_image.get_fdata().shape[-1]
    
#     array.append([flo_z_slice, ref_z_slice, ref_z_contour_slice])

#     for count, i in enumerate(indexes):
#         ref_z_slice = yield_correct_slice(ref_image, i)
#         flo_z_slice = yield_correct_slice(flo_image, i)
#         ref_z_contour_slice = yield_correct_slice(ref_contour_image, i)
#         slice_contour_array[count,:,:,]
        
#     n_contour_slices+=number_of_ref_contour_slices
#     print('number of slices containing contours:', n_contour_slices)
    print('flo:  {}  ref:  {}'.format(flo_str, ref_str))
    print(f,r,number_of_flo_img_slices,number_of_ref_img_slices)
#     print(number_of_ref_img_slices, number_of_ref_contour_slices, number_of_flo_img_slices, number_of_flo_contour_slices, contour_count)
    array.append([flo_int,ref_int,number_of_flo_img_slices,number_of_ref_img_slices])
    it+=1
# print(np.array(array).shape)
# print(np.array(array))


In [ ]:
# Find the number of floating and reference images present for the pairs which created constant/ invalid DVFs
organ='PAROTID_LT'
fr_rt = np.load('{}_flo_ref_number_of_flo_ref_3D_slices.npy'.format(organ))
print(fr_rt[:10])
flo_ref_rt = np.insert(fr_rt, 4, rt[:,-1], axis=1)

condition=0
ratio_arr=[]

for i in flo_ref_rt:
    ratio = i[3]/i[2]
    ratio_arr.append(ratio)
    if ratio>1.5 or ratio<0.5:
        condition+=i[-1]
    
print(condition,'/',sum(flo_ref_rt[:,-1]))
print('{:.2f} %'.format(100*condition/sum(flo_ref_rt[:,-1])))

plt.hist(ratio_arr, bins=30)
plt.xlabel('number of reference to floating slices ratio')
plt.ylabel('pair frequency')
plt.show()

invalid_vi = np.load('PAROTID_LT_validate_invalid_DVF_slice_indexes.npy', mmap_mode='r')
invalid_ti = np.load('PAROTID_LT_train_invalid_DVF_slice_indexes.npy', mmap_mode='r')

if organ=='PAROTID_LT':
    n_train_max = 13394
    n_validate_max = 7762
    
entire_invalid_idxs = np.concatenate((invalid_vi,invalid_ti+n_validate_max))

def find_invalid_flo_ref(rt_array, idx):
    slice_sum=0 
    for count, i in enumerate(rt_array):
        if idx+1>slice_sum:
            slice_sum+=i[-1]
        else:
            rt_of_interest = rt_array[count-1]
            break
    return rt_of_interest


flo_ref_pairs=[]
for i in range(len(entire_invalid_idxs)):
    flo_ref_pairs.append(find_invalid_flo_ref(flo_ref_rt, entire_invalid_idxs[i]))#entire_invalid_idxs[i]))

invalid_flo_ref_pairs = np.unique(flo_ref_pairs, axis=0)
x = invalid_flo_ref_pairs[:,2]
y = invalid_flo_ref_pairs[:,3]
plt.hist(y/x, bins=30) 
plt.xlabel('number of reference to floating slices ratio for invalid DVFs')
plt.ylabel('pair frequency')
plt.show()
print(invalid_flo_ref_pairs[:, 0:2])

In [ ]:
def view_slices(TorV, L, N, fname):
    imdata = np.load(fname, mmap_mode='r')
    print(len(imdata))

    for i in range(len(imdata[L:N])):
        x = imdata[i]
        print(i, TorV)
        min_x = int(256/2); max_x = int(512-256/2)
        min_y = int(256/2); max_y = int(512-256/2)
        
        image = x[min_x:max_x, min_y:max_y, 0]
        print(np.min(image),np.max(image))
        image = image.tolist()

        dvf = x[min_x:max_x, min_y:max_y,1:]
        step = 25
        dvf_x = dvf[::step,::step,0]
        dvf_y = dvf[::step,::step,1]
#         dvf_z = dvf[::step,::step,2]

        mesh_x, mesh_y = np.meshgrid(range(256),range(256))
        norm = plt.Normalize()
        colors = norm(dvf_x)
        #     colormap=plt.cm.viridis

        plt.quiver(mesh_x[::step,::step], mesh_y[::step,::step], dvf_x.tolist(), dvf_y.tolist(), color='r')
        plt.xlabel('x')
        plt.ylabel('y', rotation=0)
        plt.imshow(image, interpolation='nearest', cmap='viridis')
#         plt.show()
        
k=100; m=1
organ = 'BRAIN_STEM'; organ2 = 'PAROTID_LT'
# fname = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/complete_'+organ+TorV+'_slices_DVF_xy.npy'
fname = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/'+organ+'/complete_arrays/'\
+ 'COMPLETE_' + organ + '_multiple_patient_image_array.npy'

fname2 = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/'+organ2+'/complete_arrays/'\
+ 'COMPLETE_' + organ2 + '_multiple_patient_image_array.npy'

view_slices('validation', k, k+m, fname)
view_slices('training', k, k+m, fname2)

In [ ]:
train_validate='train' # 'validate'
train_test='train_even'
# fname = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/X_'+train_validate+'_memmap_MSD_3D.npy'

idxs='0-544'
fname = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/BRAIN_STEM/'+train_test+\
'/image_array_3D_'+train_test+'_BRAIN_STEM_'+idxs+'.npy'
imdata = np.load(fname, mmap_mode='r')
print(len(imdata))

for i in range(len(imdata[:10])):
    x = imdata[i]
    print(i)
    image = x[:,:,0]
    image = image.tolist()

    dvf = x[:,:,1:]
    step = 25
    dvf_x = dvf[::step,::step,0]
    dvf_y = dvf[::step,::step,1]
    dvf_z = dvf[::step,::step,2]

    mesh_x, mesh_y = np.meshgrid(range(512),range(512))
    norm = plt.Normalize()
    colors = norm(dvf_z)
    #     colormap=plt.cm.viridis

    plt.quiver(mesh_x[::step,::step], mesh_y[::step,::step], dvf_x.tolist(), dvf_y.tolist(), color='r')
    plt.xlabel('x')
    plt.ylabel('y', rotation=0)
    plt.imshow(image, interpolation='nearest', cmap='viridis')
    plt.show()

In [ ]:
from matplotlib.colors import ListedColormap

organ = 'PAROTID_LT'
train_test = 'train_odd'

slice_array_folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/'\
'slice_arrays/new_registrations/{}/complete_arrays/'.format(organ)

reg_t_fp = '{}_{}_registration_track.npy'.format(train_test,organ)

regT = np.load(slice_array_folder+reg_t_fp)

print(regT[:20])

for i in regT:
    flo_int=int(i[0])
    ref_int=int(i[1])

    ref_str = str(ref_int) if ref_int>=100 else '0' + str(ref_int)
    if ref_int<10:
        ref_str = '00' + str(ref_int)
    flo_str = str(flo_int) if flo_int>=100 else '0' + str(flo_int)
    if flo_int<10:
        flo_str = '00' + str(flo_int)
        
#     flo = '248'
#     ref = '839'
    flo, ref = flo_str, ref_str
    print(flo, ref)

    fr = 'Flo_{}_Ref_{}'.format(flo,ref)

    patient_img_base = '/hepgpu9-data1/sbridger/94_patient_data/patient_filepath/'
    reg_fp_base = '/hepgpu9-data1/sbridger/Auto_Registration/registration_store/new_registration/{}/'.format(train_test)

    reg_fp = reg_fp_base+'{}/Def_{}.nii'.format(fr,fr)
    reg_image = nib.load(reg_fp)
    number_of_reg_img_slices = reg_image.get_fdata().shape[-1]

    reg_contour_fp = reg_fp_base+'{}/Def_{}_{}.nii.gz'.format(fr,fr,organ)
    reg_contour_image = nib.load(reg_contour_fp)
    number_of_reg_contour_slices = reg_contour_image.get_fdata().shape[-1]

    ref_img_fp = patient_img_base+'{}/NoBackground_0522c0{}.nii.gz'.format(ref, ref)
    ref_image = nib.load(ref_img_fp)
    number_of_ref_img_slices = ref_image.get_fdata().shape[-1]

    ref_contour_fp = patient_img_base+'{}/{}.nii.gz'.format(ref,organ)
    ref_contour_image = nib.load(ref_contour_fp)
    number_of_ref_contour_slices = ref_contour_image.get_fdata().shape[-1]

    flo_img_fp = patient_img_base+'{}/NoBackground_0522c0{}.nii.gz'.format(flo, flo)
    flo_image = nib.load(flo_img_fp)
    number_of_flo_img_slices = flo_image.get_fdata().shape[-1]

    flo_contour_fp = patient_img_base+'{}/{}.nii.gz'.format(flo, organ)
    flo_contour_image = nib.load(flo_contour_fp)
    number_of_flo_contour_slices = flo_contour_image.get_fdata().shape[-1]

    # print(number_of_img_slices,number_of_ref_contour_slices, number_of_deformed_contour_slices, number_of_floating_contour_slices)
    # plt.imshow(image.get_fdata()[:,:,20])

    def show_xy_slices(data, n_slices, steps):
        for i in np.arange(0, n_slices, steps):
            if i<n_slices:
    #             Must contain contour in image
                if np.any(data.get_fdata()[:,:,i])==True: 
                    print('Slice number {}'.format(i))
                    plt.imshow(data.get_fdata()[:,:,i], interpolation='none')
                    plt.xlabel('x')
                    plt.ylabel('y', rotation=0)
                    plt.show()
            else:
                break

    # show_xy_slices(image, number_of_img_slices, 1)
    # show_xy_slices(ref_contour_image, number_of_ref_contour_slices, 1)
    # show_xy_slices(deformed_contour_image, number_of_deformed_contour_slices, 1)
    # show_xy_slices(floating_contour_image, number_of_floating_contour_slices, 1)

    def show_xy_slice_pairs(data1, data2, n_slices, steps):
        for i in np.arange(0, n_slices, steps):
            if i<n_slices:
    #             Must contain contour in image
    #             print([np.any(data1.get_fdata()[:,:,i]), np.any(data2.get_fdata()[:,:,i])])
    #             Replace and with or for def floating contour pair
                if np.any(data1.get_fdata()[:,:,i])==True and np.any(data2.get_fdata()[:,:,i])==True: 
                    print('Original slice number {}'.format(i))
                    plt.imshow(data1.get_fdata()[:,:,i], interpolation='none')
                    plt.show()
                    print('Deformed slice number {}'.format(i))
                    plt.imshow(data2.get_fdata()[:,:,i], interpolation='none')
                    plt.xlabel('x')
                    plt.ylabel('y', rotation=0)
                    plt.show()
            else:
                break

    # show_xy_slice_pairs(image, ref_contour_image, number_of_img_slices, 1)
    # show_xy_slice_pairs(image, deformed_contour_image, number_of_img_slices, 1)

    # show_xy_slice_pairs(ref_contour_image, deformed_contour_image, number_of_img_slices, 1)

    # show_xy_slice_pairs(deformed_contour_image, floating_contour_image, number_of_deformed_contour_slices, 1)

    def add_contour_to_slice(img_data, contour_data, steps, cut):
        for i in range(0, img_data.get_fdata().shape[-1], steps):
            img_data_array=img_data.get_fdata()[cut:-cut,cut:-cut, i]
            contour_data_array=contour_data.get_fdata()[cut:-cut,cut:-cut, i]
    #             Must contain contour in image
    #             print([np.any(data1.get_fdata()[:,:,i]), np.any(data2.get_fdata()[:,:,i])])
    #             Replace and with or for def floating contour pair
            if np.any(img_data_array)==True and np.any(contour_data_array)==True: 
    #             print('Original slice number {}'.format(i))
    #             plt.imshow(img_data_array, interpolation='none')
    #             plt.show()
    #             print('Deformed slice number {}'.format(i))
    #             plt.imshow(contour_data_array, interpolation='none')
    #             plt.show()
                print('Joined slice number {}'.format(i))
                value=-1
                joined_data=np.where(contour_data_array!=0, value, img_data_array)
                masked_array = np.ma.masked_where(joined_data == value, joined_data)
                cmap = plt.cm.viridis  # Default is viridis
                cmap.set_bad(color='white')
                plt.imshow(masked_array, interpolation='none', cmap=cmap) #[cut:-cut,cut:-cut]
                plt.xlabel('x')
                plt.ylabel('y', rotation=0)
                plt.show()
                
    add_contour_to_slice(flo_image, flo_contour_image, 1, 64)
#     add_contour_to_slice(ref_image, ref_contour_image, 1, 64)
#     add_contour_to_slice(reg_image, reg_contour_image, 1, 64)
    



In [ ]:
# The zeroth column represents the pixel brightness e.g.  imdata[0,:,:,0]
# The dvf values are stored in imdata[0,:,:,i] where i=1,2,3 of the (512,512,4) arrays
# 248 845: slice 111 (14th) flo, slice 98 (11th) ref
                
def add_single_contour_to_dvf_slice(img_data, dvf_data, contour_data, cut):
    for i in range(len(dvf_data)):
        img_data_array = img_data.get_fdata()[cut:-cut,cut:-cut, 80]
        contour_data_array = contour_data.get_fdata()[cut:-cut,cut:-cut, 80]
    #     print(type(img_data_array), type(contour_data_array))
    #         print(img_data_array.shape, contour_data_array.shape)
    #     print('Original slice number')
        print('Deformed contour slice number {}'.format(i))
        plt.imshow(contour_data_array, interpolation='none')
        plt.show()
        print('Joined image')
        value = -1
        joined_data = np.where(contour_data_array!=0, value, img_data_array)
        masked_array = np.ma.masked_where(joined_data == value, joined_data)
        masked_array=np.array(masked_array)

        cmap = copy.copy(plt.cm.get_cmap("viridis"))  # Default is viridis
        cmap.set_under(color='white')

        x = dvf_data[i]
        print(x.shape)
        image = masked_array.tolist()

        dvf = x[:,:,1:]
        step = 25
        dvf_x = dvf[::step,::step,0]
        dvf_y = dvf[::step,::step,1]
        dvf_z = dvf[::step,::step,2]

        mesh_x, mesh_y = np.meshgrid(range(512),range(512))
    #     norm = plt.Normalize()
    #     colors = norm(dvf_z)
        #     colormap=plt.cm.viridis

        plt.quiver(mesh_x[::step,::step], mesh_y[::step,::step], dvf_x.tolist(), dvf_y.tolist(), color='r')
        plt.xlabel('x')
        plt.ylabel('y', rotation=0)
        plt.imshow(image, interpolation='none', cmap=cmap, vmin=0)
        plt.show()
    
organ = 'PAROTID_LT'
patient_img_base = '/hepgpu9-data1/sbridger/94_patient_data/patient_filepath/'

ref = '845'

# ref_contour_fp = patient_img_base+'{}/{}.nii.gz'.format(ref, organ)
# ref_contour_image = nib.load(ref_contour_fp)


flo = '002'

slice_array_folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/'\
'slice_arrays/new_registrations/{}/complete_arrays/'.format(organ)
print('1')
flo_contour_fp = patient_img_base+'{}/{}.nii.gz'.format(flo, organ)
flo_contour_image = nib.load(flo_contour_fp)
print('1')
flo_img_fp = patient_img_base+'{}/NoBackground_0522c0{}.nii.gz'.format(flo, flo)
flo_image = nib.load(flo_img_fp)
print('1')
fr = 'Flo_{}_Ref_{}'.format(flo,ref)
train_test = 'train_odd'
# reg_fp_base = '/hepgpu9-data1/sbridger/Auto_Registration/registration_store/new_registration/{}/'.format(train_test)

# dvf = nib.load(reg_fp_base+fr+'/Disp_Vector_Field.nii').get_fdata()[98]

dvf = np.load(slice_array_folder+'train_odd_PAROTID_LT_multiple_patient_image_dvf_array.npy', mmap_mode ='c')[:,:,:,:]
    
add_single_contour_to_dvf_slice(flo_image, dvf, flo_contour_image, 64)


In [ ]:
train_test='train_even' # 'test_odd' 'COMPLETE'
fname = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/BRAIN_STEM/complete_arrays/'\
+train_test+'_BRAIN_STEM_multiple_patient_image_array.npy'

imdata = np.load(fname, mmap_mode='r')

for i in range(len(imdata)):
    x = imdata[i]
    print(i)
    image = x[:,:,0]
    image = image.tolist()

    dvf = x[:,:,1:]
    step = 25
    dvf_x = dvf[::step,::step,0]
    dvf_y = dvf[::step,::step,1]
    dvf_z = dvf[::step,::step,2]

    mesh_x, mesh_y = np.meshgrid(range(512),range(512))
    norm = plt.Normalize()
    colors = norm(dvf_z)
#     colormap=plt.cm.inferno

    plt.quiver(mesh_x[::step,::step], mesh_y[::step,::step], dvf_x.tolist(), dvf_y.tolist(), color='r')
    plt.xlabel('x')
    plt.ylabel('y', rotation=0)
    plt.imshow(image, interpolation='nearest')
    plt.show()

In [ ]:
# 
def n_ancestors(n_generations, generation_time_years, n_children):
    number=0
    for g in range(1,n_generations+1):
        number += 2**(g/n_children)
    time = n_generations*generation_time_years
    out = 'You had {:e} ancestors over {} years'.format(number, time)
    return out, number

print_, temp_n = n_ancestors(2000, 25, 66.564913)
actual_n = temp_n/(8*10**9)
print(actual_n)
n_ancestors(2000,25, actual_n)
# 2000 generations of humans